In [32]:
# YOUR CODE HERE
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pltlab
import seaborn as sns
from sklearn.model_selection import train_test_split
import calendar
import geopandas as gpd
import censusdata
import warnings
import pipeline as pipe
warnings.filterwarnings('ignore')

%matplotlib inline

sns.set(rc={'figure.figsize':(11, 4)})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
data = pd.read_pickle('data/final_dataset.pk1')

In [34]:
data.head()

,name,total_pop,median_income,state,county,county_name,state_name,prop_white,prop_black,prop_hisp,...,Testing_Rate,gov_party,election_diff,Apr-19,Mar-19,Feb-20,Mar-20,days_closed,yearly_change,monthly_change
0,"Washington County, Mississippi",47086,30834.0,28,151,Washington,Mississippi,0.256913,0.721701,0.015482,...,2291.374085,1,-0.364695,6.4,7.5,7.4,7.0,17.0,-0.066667,-0.054054
1,"Perry County, Mississippi",12028,39007.0,28,111,Perry,Mississippi,0.787745,0.196874,0.015048,...,2291.374085,1,0.536635,5.7,6.2,6.8,6.3,17.0,0.016129,-0.073529
2,"Choctaw County, Mississippi",8321,37203.0,28,019,Choctaw,Mississippi,0.676722,0.311982,0.003966,...,2291.374085,1,0.386224,4.8,5.3,5.4,5.0,17.0,-0.056604,-0.074074
3,"Itawamba County, Mississippi",23480,40510.0,28,057,Itawamba,Mississippi,0.909114,0.071593,0.015332,...,2291.374085,1,0.755161,4.1,4.4,4.7,4.7,17.0,0.068182,0.000000
4,"Carroll County, Mississippi",10129,43060.0,28,015,Carroll,Mississippi,0.643992,0.345839,0.002863,...,2291.374085,1,0.383321,5.5,5.8,6.2,6.0,17.0,0.034483,-0.032258


In [35]:
pipe.get_summary_stats(data)

total_pop  median_income   prop_white   prop_black    prop_hisp  \
count  2.826000e+03    2825.000000  2826.000000  2826.000000  2826.000000   
mean   1.117590e+05   51714.241416     0.827329     0.098748     0.091962   
std    3.414925e+05   13821.224638     0.164131     0.149708     0.135249   
min    4.180000e+02   20188.000000     0.093534     0.000000     0.000000   
25%    1.412200e+04   42491.000000     0.757614     0.008110     0.021500   
50%    3.013550e+04   49936.000000     0.889050     0.028182     0.041568   
75%    7.725500e+04   57886.000000     0.947572     0.117748     0.095686   
max    1.009805e+07  136268.000000     0.997743     0.874123     0.990688   

       log_med_income  prop_no_internet      prop_ba  prop_services  \
count     2825.000000       2826.000000  2826.000000    2825.000000   
mean        10.821023          0.225514     0.217787       0.083571   
std          0.251809          0.086132     0.096556       0.034479   
min          9.912844          0

In [41]:
train, test = train_test_split(data, random_state=246012349, test_size=0.2)
print(len(train))
print(len(test))

2260
566


In [43]:
train.dtypes
numeric_cols = train.select_dtypes(include=['float64']).columns.drop(['FIPS'])
train, test = pipe.impute_missing(train, test, numeric_cols)

In [44]:
train, test = pipe.normalize(train, test, numeric_cols)

In [46]:
train.describe()

,total_pop,median_income,prop_white,prop_black,prop_hisp,log_med_income,prop_no_internet,prop_ba,prop_services,pop_density,...,Testing_Rate,gov_party,election_diff,Apr-19,Mar-19,Feb-20,Mar-20,days_closed,yearly_change,monthly_change
count,2.260000e+03,2.260000e+03,2.260000e+03,2.260000e+03,2.260000e+03,2.260000e+03,2.260000e+03,2.260000e+03,2.260000e+03,2.260000e+03,...,2.260000e+03,2260.000000,2.260000e+03,2.260000e+03,2.260000e+03,2.260000e+03,2.260000e+03,2.260000e+03,2.260000e+03,2.260000e+03
mean,1.088654e+05,-1.823517e-16,3.599874e-16,7.859986e-17,3.143994e-18,8.504505e-16,7.152587e-17,1.886397e-17,1.689897e-16,-1.886397e-17,...,-1.634877e-16,0.556637,3.772793e-17,-4.150073e-16,6.602388e-17,1.045378e-16,-5.061831e-16,-1.705617e-16,2.986795e-17,-1.100398e-16
std,2.674309e+05,1.000221e+00,1.000221e+00,1.000221e+00,1.000221e+00,1.000221e+00,1.000221e+00,1.000221e+00,1.000221e+00,1.000221e+00,...,1.000221e+00,0.496892,1.000221e+00,1.000221e+00,1.000221e+00,1.000221e+00,1.000221e+00,1.000221e+00,1.000221e+00,1.000221e+00
min,4.590000e+02,-2.273373e+00,-4.512140e+00,-6.531657e-01,-6.802470e-01,-3.592865e+00,-2.279322e+00,-1.682501e+00,-2.477060e+00,-1.610866e-01,...,-1.051838e+00,0.000000,-3.799412e+00,-1.741680e+00,-1.882940e+00,-1.683185e+00,-1.832830e+00,-2.709177e+00,-2.540270e+00,-2.588353e+00
25%,1.416950e+04,-6.667247e-01,-4.167850e-01,-5.989420e-01,-5.217171e-01,-6.498845e-01,-6.978496e-01,-7.084033e-01,-5.997382e-01,-1.482970e-01,...,-6.081362e-01,0.000000,-5.422928e-01,-6.628759e-01,-6.517974e-01,-6.439991e-01,-6.475640e-01,-3.086970e-01,-7.589709e-01,-7.415034e-01
50%,3.038850e+04,-1.271166e-01,3.794700e-01,-4.689987e-01,-3.713675e-01,-7.782835e-03,-1.199927e-01,-2.616421e-01,-1.219710e-01,-1.324702e-01,...,-3.473213e-01,1.000000,2.130832e-01,-2.313542e-01,-2.208975e-01,-2.398711e-01,-1.837642e-01,1.966672e-01,-6.605347e-02,-5.594010e-03
75%,7.944950e+04,4.427878e-01,7.298116e-01,1.191826e-01,3.290488e-02,5.733211e-01,5.772554e-01,4.485360e-01,4.136571e-01,-8.327267e-02,...,4.255450e-01,1.000000,7.547766e-01,4.159284e-01,4.562310e-01,3.951871e-01,4.346357e-01,7.020313e-01,6.639279e-01,6.233518e-01
max,4.602523e+06,6.090030e+00,1.037609e+00,5.254312e+00,6.549744e+00,3.959295e+00,5.292440e+00,5.394969e+00,9.063082e+00,4.061721e+01,...,4.767812e+00,1.000000,1.993568e+00,8.686761e+00,9.997586e+00,8.939607e+00,9.040700e+00,1.460078e+00,4.526804e+00,5.927676e+00
